In [1]:
#参考 https://blog.csdn.net/Blank_spaces/article/details/106398162?utm_medium=distribute.pc_relevant.none-task-blog-OPENSEARCH-4.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-4.control

In [5]:
#数据集下载
# !wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
# !gzip -d reviews_Electronics_5.json.gz
# !wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
# !gzip -d meta_Electronics.json.gz

In [52]:
#review是用户的行为数据 meta是广告数据
review_sample = {
  "reviewerID": "A2SUAM1J3GNN3B", #用户id
  "asin": "0000013714",
  "reviewerName": "J. McDonald",
  "helpful": [2, 3],
  "reviewText": "I bought this for my husband who plays the piano.  He is having a wonderful time playing these old hymns.  The music  is at times hard to read because we think the book was published for singing from more than playing from.  Great purchase though!",
  "overall": 5.0,
  "summary": "Heavenly Highway Hymns",
  "unixReviewTime": 1252800000, #时间戳
  "reviewTime": "09 13, 2009"
}
meta_sample = {
  "asin": "0000031852", #物品id
  "title": "Girls Ballet Tutu Zebra Hot Pink",
  "price": 3.17,
  "imUrl": "http://ecx.images-amazon.com/images/I/51fAmVkTbyL._SY300_.jpg",
  "related":
  {
    "also_bought": ["B00JHONN1S", "B002BZX8Z6", "B00D2K1M3O", "0000031909", "B00613WDTQ", "B00D0WDS9A", "B00D0GCI8S", "0000031895", "B003AVKOP2", "B003AVEU6G", "B003IEDM9Q", "B002R0FA24", "B00D23MC6W", "B00D2K0PA0", "B00538F5OK", "B00CEV86I6", "B002R0FABA", "B00D10CLVW", "B003AVNY6I", "B002GZGI4E", "B001T9NUFS", "B002R0F7FE", "B00E1YRI4C", "B008UBQZKU", "B00D103F8U", "B007R2RM8W"],
    "also_viewed": ["B002BZX8Z6", "B00JHONN1S", "B008F0SU0Y", "B00D23MC6W", "B00AFDOPDA", "B00E1YRI4C", "B002GZGI4E", "B003AVKOP2", "B00D9C1WBM", "B00CEV8366", "B00CEUX0D8", "B0079ME3KU", "B00CEUWY8K", "B004FOEEHC", "0000031895", "B00BC4GY9Y", "B003XRKA7A", "B00K18LKX2", "B00EM7KAG6", "B00AMQ17JA", "B00D9C32NI", "B002C3Y6WG", "B00JLL4L5Y", "B003AVNY6I", "B008UBQZKU", "B00D0WDS9A", "B00613WDTQ", "B00538F5OK", "B005C4Y4F6", "B004LHZ1NY", "B00CPHX76U", "B00CEUWUZC", "B00IJVASUE", "B00GOR07RE", "B00J2GTM0W", "B00JHNSNSM", "B003IEDM9Q", "B00CYBU84G", "B008VV8NSQ", "B00CYBULSO", "B00I2UHSZA", "B005F50FXC", "B007LCQI3S", "B00DP68AVW", "B009RXWNSI", "B003AVEU6G", "B00HSOJB9M", "B00EHAGZNA", "B0046W9T8C", "B00E79VW6Q", "B00D10CLVW", "B00B0AVO54", "B00E95LC8Q", "B00GOR92SO", "B007ZN5Y56", "B00AL2569W", "B00B608000", "B008F0SMUC", "B00BFXLZ8M"],
    "bought_together": ["B002BZX8Z6"]
  },
  "salesRank": {"Toys & Games": 211836},
  "brand": "Coxlures",
  "categories": [["Sports & Outdoors", "Other Sports", "Dance"]] #种类列表
}

In [11]:
#json格式转为pickle格式
import pickle
import pandas as pd
def to_df(file_path):
    with open(file_path, 'r') as fin:
        df = {}
        i = 0
        for line in fin:
            df[i] = eval(line)
            i += 1
        df = pd.DataFrame.from_dict(df, orient='index')
        return df
reviews_df = to_df("./reviews_Electronics_5.json")

with open("reviews.pkl", 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL)

meta_df = to_df('./meta_Electronics.json')
meta_df = meta_df[meta_df['asin'].isin(reviews_df['asin'].unique())]
meta_df = meta_df.reset_index(drop=True)
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta_df, f, pickle.HIGHEST_PROTOCOL)

In [6]:
#对revies和meta数据进行处理
import random
import pickle
import numpy as np
import pandas as pd
random.seed(2020)
def build_map(df, col_name):
    key = sorted(df[col_name].unique().tolist())
    m = dict(zip(key, range(len(key))))
    df[col_name] = df[col_name].map(lambda x: m[x])
    return m, key

In [7]:
reviews_df = pd.read_pickle('./reviews.pkl')
#评论只抽取用户id, 商品id, 时间戳
review_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
meta_df = pd.read_pickle('./meta.pkl')
meta_df = meta_df[['asin', 'categories']]
meta_df['categories'] = meta_df['categories'].map(lambda x: x[-1][-1])

In [9]:
#meta_df的物品id映射
asin_map, asin_key = build_map(meta_df, 'asin')
cate_map, cate_key = build_map(meta_df, 'categories')
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

In [10]:
user_count, item_count, cate_count, example_count = len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]
print (user_count)
#按物品id排序，并重置索引
meta_df = meta_df.sort_values('asin')
meta_df = meta_df.reset_index(drop=True)
#review_df文件物品id进行映射，并按照用户id、浏览时间进行排序，充值索引
reviews_df['asin'] = review_df['asin'].map(lambda x: asin_map[x])
reviews_df = reviews_df.sort_values(['reviewerID', 'unixReviewTime'])
reviews_df = reviews_df.reset_index(drop=True)
reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
#各个物品对应的类别
cate_list = np.array(meta_df['categories'], dtype='int32')

#保存所需数据为pkl文件
with open('./remap.pkl', 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, cate_count, example_count), f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((asin_key, cate_key, revi_key), f, pickle.HIGHEST_PROTOCOL)

192403


In [11]:
from tqdm import tqdm_notebook as tqdm
t = (reviews_df['reviewerID'].unique().tolist())
print (len(t))

192403


In [12]:
with open('remap.pkl', 'rb') as f:
    reviews_df = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count, example_count = pickle.load(f)

train_set, test_set = [], []
max_sl = 0    
for reviewerID, hist in reviews_df.groupby('reviewerID'):
    # 每个用户浏览过的物品，即为正样本
    pos_list = hist['asin'].tolist()
    max_sl = max(max_sl, len(pos_list))
    # 生成负样本
    def gen_neg():
        neg = pos_list[0]
        while neg in pos_list:
            neg = random.randint(0, item_count - 1)
        return neg

    # 正负样本比例1：1
    neg_list = [gen_neg() for i in range(len(pos_list))]

    for i in range(1, len(pos_list)):
        # 生成每一次的历史记录，即之前的浏览历史
        hist = pos_list[:i]
        sl = len(hist)
        if i != len(pos_list) - 1:
            # 保存正负样本，格式：用户ID，正/负物品id，浏览历史，浏览历史长度，标签（1/0）
            train_set.append((reviewerID, pos_list[i], hist, sl, 1))
            train_set.append((reviewerID, neg_list[i], hist, sl, 0))
        else:
            # 最后一次保存为测试集
            test_set.append((reviewerID, pos_list[i], hist, sl, 1))
            test_set.append((reviewerID, neg_list[i], hist, sl, 0))

# 打乱顺序
random.shuffle(train_set)
random.shuffle(test_set)
print (len(test_set), user_count)
assert len(test_set) == 2 * user_count
# 写入dataset.pkl文件
with open('./dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, cate_count, max_sl), f, pickle.HIGHEST_PROTOCOL)

384806 192403


In [22]:
import tensorflow as tf
class DIN(tf.keras.Model):
    def __init__(self, user_num, item_num, cate_num, cate_list, hidden_units):
        """
        user_num: 用户数量
        item_num: 物品数量
        cate_num: 物品种类数量
        cate_list: 物品种类列表
        hidden_units: 隐藏层单元
        """
        super(DIN, self).__init__()
        self.cate_list = tf.convert_to_tensor(cate_list, dtype=tf.int32)
        self.hidden_units = hidden_units
        self.item_embed = tf.keras.layers.Embedding(
            input_dim = item_num, output_dim=self.hidden_units, embeddings_initializer='random_uniform',
            embeddings_regularizer = tf.keras.regularizers.l2(0.01), name='item_embed'
            )
        self.cate_embed = tf.keras.layers.Embedding(
            input_dim = cate_num, output_dim=self.hidden_units, embeddings_initializer='random_uniform',
            embeddings_regularizer = tf.keras.regularizers.l2(0.01), name='cate_embed'
            )
        self.dense = tf.keras.layers.Dense(self.hidden_units)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.att_dense1 = tf.keras.layers.Dense(80, activation='sigmoid')
        self.att_dense2 = tf.keras.layers.Dense(40, activation='sigmoid')
        self.att_dense3 = tf.keras.layers.Dense(1)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.concat2 = tf.keras.layers.Concatenate(axis=-1)
        self.dense1 = tf.keras.layers.Dense(80, activation='sigmoid')
        self.activation1 = tf.keras.layers.PReLU()
        # self.activation1 = Dice()
        self.dense2 = tf.keras.layers.Dense(40, activation='sigmoid')
        self.activation2 = tf.keras.layers.PReLU()
        # self.activation2 = Dice()
        self.dense3 = tf.keras.layers.Dense(1, activation=None)
    def call(self, inputs):
        #user为用户id， item为物品id， hist为之前的历史记录，即物品id列表，sl为最大列表长度
        user, item, hist, sl = inputs[0], tf.squeeze(inputs[1], axis=1), inputs[2], tf.squeeze(inputs[3], axis=1)
        item_embed = self.concat_embed(item)
        print (item_embed)
        hist_embed = self.concat_embed(hist)
        print (hist_embed)
        #经过attention的物品embedding
        hist_att_embed = self.attention(item_embed, hist_embed, sl)
        hist_att_embed = self.bn1(hist_att_embed)
        hist_att_embed = tf.reshape(hist_att_embed, [-1, self.hidden_units * 2])
        u_embed = self.dense(hist_att_embed)
        item_embed = tf.reshape(item_embed, [-1, item_embed.shape[-1]])
        #联合用户行为embedding， 候选物品embedding
        embed = self.concat2([u_embed, item_embed])
        #mlp
        x = self.bn2(embed)
        x = self.dense1(x)
        x = self.activation1(x)
        x = self.dense2(x)
        x = self.activation2(x)
        x = self.dense3(x)
        outputs = tf.nn.sigmoid(x)
        return outputs
        
    def concat_embed(self, item):
        #拼接物品embedding和物品种类embedding
        #input : 物品id
        #return: 拼接后的embedding
        cate = tf.gather(self.cate_list, item)
        cate = tf.squeuze(cate, axis=1) if cate.shape[-1] == 1 else cate
        item_embed = self.item_embed(item)
        item_cate_embed = self.cate_embed(cate)
        embed = self.concat([item_embed, item_cate_embed])
        return embed
    def attention(self, queries, keys, keys_length):
        """
        activate unit
        params queries: 候选广告（物品）embedding
        param keys: 用户行为（历史记录） embedding
        param keys_length: 用户行为embedding中的有效长度
        return
        """
        #候选物品的隐藏向量维度 hidden_unit * 2
        queries_hidden_units = queries.shape[-1]
        #每个历史记录的物品的embed都要与候选物品的embed拼接，顾候选物品embed重复keys.shape[1]次
        #keys.shape[1]为最大的序列长度，即431, 为了方便矩阵计算
        # [None, 431 * hidden_unit * 2]
        queries = tf.tile(queries, [1, keys.shape[1]])
        #重塑候选物品embed的shape
        # [None, 431, hidden_unit * 2]
        queries = tf.reshape(queries, [-1, keys.shape[1], queries_hidden_units])
        #拼接候选物品embed与hist物品embed
        # [None, 431 * 2 * 4]
        embed = tf.concat([queries, keys, queries - keys, queries * keys], axis=-1)
        #全连接， 得到权重w
        d_layer_1 = self.att_dense1(embed)
        d_layer_2 = self.att_dense2(d_layer_1)
        #[None, 431, 1]
        d_layer_3 = self.att_dense3(d_layer_2)
        #重塑输出权重类型，每个hist物品embed有对应权重值
        # [None, 1, 431]
        outputs = tf.reshape(d_layer_3, [-1, 1, keys.shape[1]])
        #Mask
        # 此处将历史记录的物品embed令为True
        # [None, 431]
        key_masks = tf.sequence_mask(keys_length, keys.shape[1])
        # 增添维度
        # [None, 1, 431]
        key_masks = tf.expand_dims(key_masks, 1)
        # 填充矩阵
        paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
        # 构造输出矩阵， 其实就是为了实现[sum_pooling]， true即为原outputs的值， False为上述填充值，为很小的值，softmax后接近0
        # [None, 1, 431] 每个历史浏览物品的权重
        outputs = tf.where(key_masks, outputs, paddings)
        # scale， keys.shape[-1]为hist_embed的隐藏单元数
        outputs = outputs / (keys.shape[-1] ** 0.5)
        # Activation, 归一化
        outputs = tf.nn.softmax(outputs)
        # 对hist_embed进行加权
        # [None, 1, 431] * [None, 431, hidden_unit * 2] = [None, 1, hidden_unit * 2]
        outputs = tf.matmul(outputs, keys)
        return outputs

In [14]:
def input_data(dataset, max_sl):
    user = np.array(dataset[:, 0], dtype='int32')
    item = np.array(dataset[:, 1], dtype='int32')
    hist = dataset[:, 2]
    hist_matrix = tf.keras.preprocessing.sequence.pad_sequences(hist, maxlen=max_sl, padding='post')
    sl = np.array(dataset[:, 3], dtype='int32')
    y = np.array(dataset[:, 4], dtype='float32')
    return user, item, hist_matrix, sl, y

In [15]:
import tensorflow as tf
from time import time
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
import os

file = 'raw_data/remap.pkl'
maxlen = 20
embed_dim = 8
att_hidden_units = [80, 40]
ffn_hidden_units = 64
dnn_dropout = 0.5
att_activation = 'sigmoid'
ffn_activation = 'prelu'

learning_rate = 0.001
batch_size = 8
epochs = 1
#========create dataset============
with open('dataset.pkl', 'rb') as f:
    train_set = pickle.load(f)
    test_set = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count, max_sl = pickle.load(f)
    train_inputs = input_data(np.array(train_set), max_sl)
    test_inputs = input_data(np.array(test_set), max_sl)


In [ ]:
#=======build model===============
model = DIN(user_count, item_count, cate_count, cate_list, ffn_hidden_units)
#model.summary()
# ============================model checkpoint======================
# check_path = 'save/din_weights.epoch_{epoch:04d}.val_loss_{val_loss:.4f}.ckpt'
# checkpoint = tf.keras.callbacks.ModelCheckpoint(check_path, save_weights_only=True,
#                                                 verbose=1, period=5)
# =========================Compile============================
model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate), metrics=[AUC()])
# ===========================Fit==============================
model.fit(
    train_inputs[0:4],
    train_inputs[4],
    epochs=epochs,
    # callbacks=[EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)],  # checkpoint
    validation_data=(test_inputs[0:3], test_inputs[3]),
    batch_size=batch_size,
)
model.summary()

# ===========================Test==============================
print('test AUC: %f' % model.evaluate(test_X, test_y, batch_size=batch_size)[1])

Tensor("din_4/concatenate_8/concat:0", shape=(None, 128), dtype=float32)
Tensor("din_4/concatenate_8/concat_1:0", shape=(None, 431, 128), dtype=float32)
Tensor("din_4/concatenate_8/concat:0", shape=(None, 128), dtype=float32)
Tensor("din_4/concatenate_8/concat_1:0", shape=(None, 431, 128), dtype=float32)
 24223/326096 [=>............................] - ETA: 1:15:39 - loss: 0.8952 - auc_4: 0.6400